In [1]:
%matplotlib inline

import os

import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency
from matplotlib import pyplot as plt
from collections import Counter

PATH_TO_DATA = '../data'

In [4]:
orange_small_churn_data = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
orange_small_churn_labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None)
orange_small_churn_labels.columns = ['lbl']

In [4]:
orange_small_churn_data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


In [5]:
orange_small_churn_labels.head()

,lbl
0,-1
1,-1
2,-1
3,1
4,-1


In [6]:
orange_small_churn_labels.lbl.value_counts()

-1    37024
 1     2976
Name: lbl, dtype: int64

### Подумайте и предложите несколько способов (не менее 3х) обработки категориальных признаков, для того, чтобы их можно было использовать при построении модели. Обратите внимание на модуль sklearn.preprocessing. Начать поиски можно с sklearn.preprocessing.OneHotEncoder.

In [6]:
cat_df = orange_small_churn_data[orange_small_churn_data.columns[-40:]]
cat_df.head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,DlISMzi,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,FoxgUHSK8h,LrdZy8QqgUfkVShG,NaN,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,vNEvyxLp3Z,RO12,NaN,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,4e7gUH7IEC,RO12,NaN,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


### Посмотрим на заполнение признаков, как в целом так и в разрезе классов

In [8]:
grouped_counts = cat_df.groupby(orange_small_churn_labels.lbl).count()
counts = orange_small_churn_labels.lbl.value_counts()

fill_counts = np.array(cat_df.count())
null_counts = np.array([cat_df.shape[0]] * cat_df.shape[1]) - fill_counts

fill_counts_outflow = grouped_counts.iloc[1].values
fill_counts_not_outflow = grouped_counts.iloc[0].values

null_counts_outflow = np.array([counts[1]] * cat_df.shape[1]) - fill_counts_outflow
null_counts_not_outflow = np.array([counts[-1]] * cat_df.shape[1]) - fill_counts_not_outflow

null_df = pd.DataFrame()
null_df['fill_counts'] = fill_counts
null_df['null_counts'] = null_counts
null_df['fill_counts_outflow'] = fill_counts_outflow
null_df['null_counts_outflow'] = null_counts_outflow
null_df['fill_counts_not_outflow'] = fill_counts_not_outflow
null_df['null_counts_not_outflow'] = null_counts_not_outflow
null_df.index = cat_df.columns

null_df.sort_values(by='fill_counts', ascending=False)

,fill_counts,null_counts,fill_counts_outflow,null_counts_outflow,fill_counts_not_outflow,null_counts_not_outflow
Var211,40000,0,2976,0,37024,0
Var210,40000,0,2976,0,37024,0
Var207,40000,0,2976,0,37024,0
Var212,40000,0,2976,0,37024,0
Var228,40000,0,2976,0,37024,0
Var204,40000,0,2976,0,37024,0
Var216,40000,0,2976,0,37024,0
Var220,40000,0,2976,0,37024,0
Var221,40000,0,2976,0,37024,0
Var198,40000,0,2976,0,37024,0


Бросяется в глаза что признаки Var191, Var213, Var215 и Var224 очень сильно разряжены.  
А так же Var209 и Var230 вообще пусты.  
Полагаю что обработку категориальных переменных в обоих вышеописаных признаках можно произвести банальным удалением :)

### Посмотрим сколько уникальных значений могут принимать признаки

In [9]:
unique_counts = []
unique_counts_outflow = []
unique_counts_not_outflow = []
intersect_counts = []

grouped = cat_df.groupby(orange_small_churn_labels.lbl)
outflow = grouped.get_group(1)
not_outflow = grouped.get_group(-1)

for c in cat_df.columns:
    unique_counts.append(cat_df[c].dropna().unique().shape[0])
    
    unique_outflow = outflow[c].dropna().unique()
    unique_not_outflow = not_outflow[c].dropna().unique()
    
    unique_counts_outflow.append(unique_outflow.shape[0])
    unique_counts_not_outflow.append(unique_not_outflow.shape[0])
    
    intersect_counts.append(np.intersect1d(unique_outflow, unique_not_outflow).shape[0])

unique_df = pd.DataFrame()
unique_df['unique_counts'] = unique_counts
unique_df['unique_counts_outflow'] = unique_counts_outflow
unique_df['unique_counts_not_outflow'] = unique_counts_not_outflow
unique_df['intersect_counts'] = intersect_counts

unique_df.index = cat_df.columns

unique_df.sort_values(by='unique_counts', ascending=False)

,unique_counts,unique_counts_outflow,unique_counts_not_outflow,intersect_counts
Var200,13297,1140,12729,572
Var214,13297,1140,12729,572
Var217,12470,1815,12064,1409
Var202,5542,1732,5485,1675
Var199,4400,1009,4072,681
Var220,3891,1141,3803,1053
Var222,3891,1141,3803,1053
Var198,3891,1141,3803,1053
Var216,1819,425,1772,378
Var192,354,277,341,264


In [22]:
unique_df.sort_values(by='unique_counts', ascending=False).index

Index(['Var200', 'Var214', 'Var217', 'Var202', 'Var199', 'Var220', 'Var222',
       'Var198', 'Var216', 'Var192', 'Var197', 'Var204', 'Var212', 'Var193',
       'Var228', 'Var226', 'Var195', 'Var219', 'Var206', 'Var207', 'Var221',
       'Var227', 'Var210', 'Var203', 'Var229', 'Var223', 'Var196', 'Var225',
       'Var205', 'Var194', 'Var211', 'Var218', 'Var201', 'Var208', 'Var191',
       'Var224', 'Var215', 'Var213', 'Var209', 'Var230'],
      dtype='object')

Можно отметить что признаки Var200, Var214, Var217,	Var202,	Var199, Var220, Var222,	Var198, Var216 могут принимать достаточно много уникалиных значений применять для них OHE может быть не самой хорошей идеей, т.к. приведет к значительному увеличинию матрици, что увеличит время обучения, но пока еще рано утверждать это однозначно.

Признаки Var191, Var224, Var215, Var213 могут принимать не так много различных значений, надо будет посмотреть в каких случаях они заполняются, но пока можно сказать что они могут оказаться потенциально бессполездными.

Так же заметно что признаки принимают не так много общих значений значений. Возможно можно заменить категории которые не встречаются в другом классе на какое-то одно значение. 

### Преобразование категориальных пременных

К тем признакам которые принимают только одно значение можно применить обычный replace и заменить наличия значения на 1 и его отсутствие на 0.

In [10]:
cat_df['Var213'].value_counts()

KdSa    890
Name: Var213, dtype: int64

In [11]:
var213_dict = {'N/A': 0, 'KdSa':1}

In [12]:
cat_df['Var213'].fillna('N/A').replace(var213_dict).head()

0    0
1    0
2    0
3    0
4    0
Name: Var213, dtype: int64

In [13]:
cat_df['Var213'].fillna('N/A').value_counts()

N/A     39110
KdSa      890
Name: Var213, dtype: int64

К тем признакам которые могут принимать среднее количество уникальных значений можно применить, например таких как Var221, можно применить подход One Hot Encording.

In [14]:
cat_df['Var221'].head(15)

0     Al6ZaUT
1        oslk
2        zCkv
3        oslk
4        oslk
5     d0EEeJi
6        oslk
7        oslk
8        oslk
9        oslk
10       oslk
11       oslk
12       zCkv
13       oslk
14       oslk
Name: Var221, dtype: object

In [15]:
pd.get_dummies(cat_df, columns=['Var221']).iloc[:, -7:].head(15)

,Var221_Al6ZaUT,Var221_JIiEFBU,Var221_QKW8DRm,Var221_d0EEeJi,Var221_oslk,Var221_z4pH,Var221_zCkv
0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0
5,0,0,0,1,0,0,0
6,0,0,0,0,1,0,0
7,0,0,0,0,1,0,0
8,0,0,0,0,1,0,0
9,0,0,0,0,1,0,0


Для признаков которые могут принимать много уникальных значений, например признак Var197, можно применить Label Encoding.

In [16]:
cat_df['Var197'].head(15)

0     0LaQ
1     YFAj
2     TyGl
3     0Xwj
4     vSNn
5     h3JW
6     0Xwj
7     dTod
8     TyGl
9     AHgj
10    EPHy
11    TyGl
12    LloI
13    ssAy
14    wOc1
Name: Var197, dtype: object

In [17]:
cat_df['Var197'].astype('category').cat.codes.head(15)

0       0
1     126
2     110
3       2
4     205
5     158
6       2
7     149
8     110
9      33
10     48
11    110
12     80
13    197
14    210
dtype: int16

In [7]:
to_label_enc = ['Var200', 'Var214', 'Var217', 'Var202', 'Var199', 'Var220', 'Var222', 'Var198', 'Var216', 'Var192', 
                'Var197', 'Var204', 'Var212', 'Var193','Var228']

for c in to_label_enc:
    cat_df['New_' + c] = cat_df[c].astype('category').cat.codes

C:\Users\Public\PythonProjects\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
to_ohe = ['Var226', 'Var195', 'Var219', 'Var206', 'Var207', 'Var221', 'Var227', 'Var210', 'Var203', 'Var229', 'Var223', 
          'Var196', 'Var225', 'Var205', 'Var194', 'Var211', 'Var218', 'Var201', 'Var208']

cat_df = pd.get_dummies(cat_df, columns=to_ohe)

In [9]:
to_bool = ['Var191', 'Var224', 'Var215', 'Var213']

for c in to_bool:
    value = cat_df[c].value_counts().index.values[0]
    value_dict = {'N/A':0, value: 1}
    cat_df['New_' + c] = cat_df[c].fillna('N/A').replace(value_dict)

In [36]:
cat_df.iloc[:, 40:].head()

,Var226_7P5s,Var226_7aLG,Var226_Aoh3,Var226_FSa2,Var226_PM2D,Var226_Qcbd,Var226_Qu4f,Var226_TNEC,Var226_WqMG,Var226_Xa3G,...,Var218_UYBR,Var218_cJvF,Var201_6dX3,Var201_smXZ,Var208_kIsH,Var208_sBgB,New_Var191,New_Var224,New_Var215,New_Var213
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0


In [10]:
num_df = orange_small_churn_data.iloc[:, :-40]

In [18]:
new_df = pd.concat([num_df, cat_df.iloc[:, 40:]], axis=1)
new_df['target'] = orange_small_churn_labels.lbl

In [57]:
new_df.head(50).to_csv(os.path.join(PATH_TO_DATA, 'new_df.train'))

In [20]:
new_df.to_csv(os.path.join(PATH_TO_DATA, 'new_df_all.train'))

### Подумайте, с помощью какой метрики качества лучше всего оценивать качество будущей модели, какой будет ключевая метрика качества? Поясните свой выбор.

Как можно заметить в выборке присутствует не так много объектов с классом отток. Так что такая метрика как accuracy будет нам бесполезна. На мой взгляд здесь подойдет метрика roc_aoc_score, которая позволяет оценивать качество классификатора на несбалансированных выборках. Так же эта метрика достаточно удобна в случае работы с вероятностями того что объект относится к какому-то классу.

### Какие вспомогательные метрики качества имеет смысл считать, чтобы более полно оценить качество модели, понять, где её слабые стороны и что нужно улучшать?

Так же если учесть что с точки зрения бизнеса "ошибка не нахождения пользователя который решил перестать пользоваться услугами компании является более критичной" можно применить метрики Precision и Recall которая позволит определить на сколько точно на сколько точно мы можем находить таких пользователей. 

<center>
$Precision = \frac{TP}{TP+FP}$
</center>

<center>
$Recall=\frac{TP}{TP+FN}$
</center>

Но тут уже надо отметить что 2 метрики не всегда удобно оптимизировать. Так что в случае если мы говорим о метриках Precision и Recall можно взять метрику F-мера которая является объединением этих метрик и считается по формуле:

<center>
$F=2\frac{Precision×Recall}{Precision+Recall}$
</center>

### Подберите оптимальную стратегию проведения кросс-валидации: решите, на сколько фолдов вы будете делить выборку? Выберите тип разбиения данных (k-fold, stratified k-fold, shuffle split, leave one out). Поясните ваш выбор.

Надо учитывать что в выборке сильный перевес класса "не отток", соответственно надо следить за тем что бы подвыборки которые получились после разбиения содержали оба класса, в обратном случае можно получить константную модель и смещенную оценку. На мой взгляд можно использовать stratified split который умеет поддерживать баланс между классами.